In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train['source'] = 'train'
test['source']='test'

In [ ]:
data = pd.concat([train,test],ignore_index=True)
data.reset_index(drop=True)

In [ ]:
print(data.shape,train.shape,test.shape)

# Data Exploration

In [ ]:
data.apply(lambda x: sum(x.isnull()))

In [ ]:
data.describe()

In [ ]:
data.apply(lambda x: len(x.unique()))

In [ ]:
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
categorical_columns = [x for x in categorical_columns if x not in ['Outlet_Identifier',
                                                                  'source']]
for col in categorical_columns:
    print('\nFrequency of categories for variable %s'%col)
    print(data[col].value_counts())

# Data Cleaning

In [ ]:
data.head(5)

In [ ]:
item_avg_weight = data.groupby('Item_Identifier').Item_Weight.mean()


In [ ]:
miss_bool = data['Item_Weight'].isnull()

In [ ]:
#Impute data and check #missing values before and after imputation to confirm
print ('Orignal #missing: %d'% sum(miss_bool))
data.loc[miss_bool,'Item_Weight'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight[x])
print ('Final #missing: %d'% sum(data['Item_Weight'].isnull()))

In [ ]:
#Outlet_Size
from scipy.stats import mode


In [ ]:
#Determing the mode for each
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x: x.mode())) 
print('Mode for each Outlet_Type: ')
print(outlet_size_mode)

#Get a boolean variable specifying missing Item_Weight values
miss_bool = data['Outlet_Size'].isnull() 

#Impute data and check #missing values before and after imputation to confirm
print('\nOrignal #missing: %d'% sum(miss_bool))
data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print('Final #missing: %d'%sum(data['Outlet_Size'].isnull()))

# Feature Engineering
1. missing values: resolved
2. combining outlet type
3. item visibility zero for some items
4. Create a broad category of Type of Item
5. Determine the years of operation of a store
4. multiple labels with same meaning in item fat content
5. Label and One Hot encoding
6. Exporting data


# Consider combining Outlet_Type

 During exploration, we decided to consider combining the Supermarket Type2 and Type3 variables. But is that a good idea? A quick way to check that could be to analyze the mean sales by type of store. If they have similar sales, then keeping them separate won’t help much.

In [ ]:
data.pivot_table('Item_Outlet_Sales','Outlet_Type')

In [ ]:
# This shows significant difference between them and we’ll leave them as it is.




# Modify Item_Visibility

In [ ]:
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')

In [ ]:
miss_bool = (data['Item_Visibility'] == 0)

In [ ]:
print('Number of 0 values initially: %d'%sum(miss_bool))

In [ ]:
data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: visibility_avg.loc[x])

In [ ]:
print('Numbers of 0 values after modification :%d'%sum(data['Item_Visibility']==0))

In [ ]:
data['Item_Visibility'].min()


But along with comparing products on absolute terms, 
we should look at the visibility of the product in that 
particular store as compared to the mean visibility of 
that product across all stores. This will give some idea 
about how much importance was given to that product in a 
store as compared to other stores. We can use the 
‘visibility_avg’ variable made above to achieve this.

In [ ]:
#Determine another variable with means ratio
data['Item_Visibility_MeanRatio'] = data.apply(lambda x: x['Item_Visibility']/visibility_avg.loc[x['Item_Identifier']], axis=1)
print(data['Item_Visibility_MeanRatio'].describe())


# Create a broad category of Type of Item

In [ ]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].rename({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

 # Modify categories of Item_Fat_Content

In [ ]:
#Change categories of low fat:

print('Original categories:')
print(data['Item_Fat_Content'].value_counts())

print('\nModified categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                            'low fat': 'Low Fat',
                                                            'reg':'Regular'})
print(data['Item_Fat_Content'].value_counts())

we saw there were some non-consumables as well and a fat-content should not be specified for them. So we can also create a separate category for such kind of observations.

In [ ]:
#Mark non-consumables as separate category in low_fat:

data.loc[data['Item_Type_Combined'] == "NC",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

# Determine the years of operation of a store

In [ ]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

# Numerical and One-Hot Coding of Categorical variables

In [ ]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [ ]:
data

In [ ]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])


In [ ]:
data.dtypes

In [ ]:
# Lets look at the 3 columns formed from Item_Fat_Content.

data[['Item_Fat_Content_0','Item_Fat_Content_1','Item_Fat_Content_2']].head()

# Exporting Data

In [ ]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [ ]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [ ]:
#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

# Model Building

In [ ]:
# baseline Model

In [ ]:
mean_sales = train['Item_Outlet_Sales'].mean()
mean_sales

In [ ]:
base1 = test[['Item_Identifier','Outlet_Identifier']]
base1['Item_Outlet_Sales'] = mean_sales

In [ ]:
#Export submission file
base1.to_csv('baseline.csv',index = False)

In [ ]:
b = pd.read_csv('baseline.csv')

In [ ]:
b.head()

a generic function which takes the algorithm and data as input and makes the model, performs 
cross-validation and generates submission.

In [ ]:
#Define target and ID columns:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn.model_selection import cross_val_score

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename, resid=False, transform=False):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    
    cv_score = cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print ("\nModel Report")
    print ("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [ ]:
# Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso

predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors

alg1 = LinearRegression(normalize=True)
modelfit(alg1, train, test, predictors, target, IDcol, 'alg1.csv')
coef1 = pd.Series(alg1.coef_, predictors).sort_values()
coef1.plot(kind='bar', title='Model Coefficients',figsize=(10,6))




In [ ]:
# Ridge Regression Model:

predictors = [x for x in train.columns if x not in [target]+IDcol]
alg2 = Ridge(alpha=0.05,normalize=True)
modelfit(alg2, train, test, predictors, target, IDcol, 'alg2.csv')
coef2 = pd.Series(alg2.coef_, predictors).sort_values()
coef2.plot(kind='bar', title='Model Coefficients',figsize=(10,6))

In [ ]:
# Decision Tree Model

from sklearn.tree import DecisionTreeRegressor
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg3 = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)
modelfit(alg3, train, test, predictors, target, IDcol, 'alg3.csv')
coef3 = pd.Series(alg3.feature_importances_, predictors).sort_values(ascending=False)
coef3.plot(kind='bar', title='Feature Importances',figsize=(10,6))

In [ ]:
# Decision Tree with only 4 variables ,max depth of 8 and min samples of 50


predictors = ['Item_MRP','Outlet_Type_0','Outlet_5','Outlet_Years']
alg4 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=150)
modelfit(alg4, train, test, predictors, target, IDcol, 'alg4.csv')
coef4 = pd.Series(alg4.feature_importances_, predictors).sort_values(ascending=False)
coef4.plot(kind='bar', title='Feature Importances')


In [ ]:
# random forest


from sklearn.ensemble import RandomForestRegressor
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg5 = RandomForestRegressor(n_estimators=200,max_depth=5, min_samples_leaf=100,n_jobs=4)
modelfit(alg5, train, test, predictors, target, IDcol, 'alg5.csv')
coef5 = pd.Series(alg5.feature_importances_, predictors).sort_values(ascending=False)
coef5.plot(kind='bar', title='Feature Importances',figsize=(10,6))


In [ ]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
alg6 = RandomForestRegressor(n_estimators=400,max_depth=6, min_samples_leaf=100,n_jobs=4)
modelfit(alg6, train, test, predictors, target, IDcol, 'alg6.csv')
coef6 = pd.Series(alg6.feature_importances_, predictors).sort_values(ascending=False)
coef6.plot(kind='bar', title='Feature Importances')